## Dependencies

In [32]:
import os
import json
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np


import torch
from torch.utils.data import DataLoader,random_split
from torchvision import transforms


import importlib
import dataloader
import config
import model
import train


importlib.reload(dataloader)
importlib.reload(config)
importlib.reload(model)
importlib.reload(train)

from train import train,validate,inference
from dataloader import CustomDataloader,train_test_split
from config import Config
from model import ModelV1


config = Config()

## Prepare Dataset

In [2]:

dataset = CustomDataloader(config.data_dir,config.transform)
train_dataset, test_dataset = train_test_split(dataset, config.test_split)

train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=config.shuffle)
test_loader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=config.shuffle)


## Create Model

In [3]:
model_obj = ModelV1()
if config.load_model:model_obj.load_state_dict(torch.load(config.model_path))
_ = model_obj.to(config.device)

## Train

In [4]:
train(model = model_obj,dataloader = train_loader,config = config)

[Train] Epoch [1/1], Batch [47/47], Loss: 0.0788  ------> Epoch [1/1] Completed. Avg Loss: 1.1627


## Validate Model

In [5]:
_ = validate(model = model_obj, dataloader = test_loader, config = Config())

[Validation] Batch [185/185] - Batch Loss: 0.0015
[Validation] Completed. Avg Loss: 0.9449

[Validation Metrics]
MSE: 0.9475
RMSE: 0.9734
MAE: 0.7754
R2: 0.0141
MAPE (%): 134.9946


## inference

In [34]:
for i in range(1,10):
    paths = f"/Users/maheshsaravanan/Documents/HemoScan/Dataset/{i:04d}"
    inference(model = model_obj, path = paths, mean = dataset.hb_mean, std = dataset.hb_std)

Predicted HB: 11.43       | Actual HB: 13.20
Predicted HB: 11.20       | Actual HB: 13.00
Predicted HB: 11.61       | Actual HB: 13.20
Predicted HB: 11.52       | Actual HB: 11.60
Predicted HB: 11.24       | Actual HB: 11.30
Predicted HB: 11.16       | Actual HB: 10.70
Predicted HB: 11.94       | Actual HB: 12.20
Predicted HB: 11.49       | Actual HB: 12.30
Predicted HB: 11.82       | Actual HB: 12.80
